<a href="https://colab.research.google.com/github/roysouvik2/PINN_optimal/blob/main/optimal1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define the PINN model
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(40, activation='tanh')  # Increased neurons
        self.dense2 = tf.keras.layers.Dense(40, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(1)

    def call(self, x, t):
        inputs = tf.concat([x, t], axis=1)
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

# Define the PDE loss for the heat equation
def pde_loss(model, x, t, alpha):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        u = model(x, t)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)
        u_t = tape.gradient(u, t)
        residual = u_t - alpha * u_xx

    loss_de = tf.reduce_mean(tf.square(residual))
    return loss_de

# Define the cost function for the optimal control problem
def cost_function(model, x, t_target, u_target):
    t_target_broadcast = tf.broadcast_to(t_target, tf.shape(x))
    u_pred = model(x, t_target_broadcast)
    loss_cost = tf.reduce_mean(tf.square(u_pred - u_target))
    return loss_cost

# Define the initial condition as a trainable variable
def initial_condition(x):
    init_cond = tf.Variable(tf.ones(x.shape), dtype=tf.float32, trainable=True)
    return init_cond

# Define the training step with gradient clipping and loss balancing
def train_step(model, x, t, x0, u0, alpha, optimizer, clip_norm, weight_pde, weight_func):
    with tf.GradientTape() as tape:
        t0 = tf.zeros_like(x0)  # time t = 0
        u_pred0 = model(x0, t0)

        #ic_loss_value = tf.reduce_mean(tf.square(u_pred0 - tf.convert_to_tensor(u0)))
        pde_loss_value = pde_loss(model, x, t, alpha)
        func_loss_value = cost_function(model, x, t_target, u_target)

        total_loss = (weight_pde * pde_loss_value +
                      weight_func * func_loss_value)

    grads = tape.gradient(total_loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, clip_norm)  # Gradient clipping
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return pde_loss_value, func_loss_value, total_loss

# Training function with learning rate scheduler and loss balancing
def train_optimal_control(model, x, t, x0, u0, alpha, u_target, t_target, epochs=3000):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1000,
        decay_rate=0.9
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    for epoch in range(epochs):
        total_pde_loss = 0
        total_ic_loss = 0
        total_func_loss = 0
        total_loss = 0

        pde_loss_value, func_loss_value, current_loss = train_step(
            model, x, t, x0, u0, alpha, optimizer,
            clip_norm = 1.0,
            weight_pde=1.0,  # Weight for PDE loss
            weight_func=10.0,  # Increased weight for functional loss
        )

        total_pde_loss += pde_loss_value.numpy()
        #total_ic_loss += ic_loss_value.numpy()
        total_func_loss += func_loss_value.numpy()
        total_loss += current_loss.numpy()

        if epoch % 1000 == 0:
            print(f'Epoch {epoch}: PDE Loss = {total_pde_loss:.3e}, '
                  f'Functional Loss = {total_func_loss:.3e}, '
                  f'Total Loss = {total_loss:.3e}')

    return model

# Generate training data
x_train = tf.random.uniform((100, 1), 0.0, 1.0)
t_train = tf.random.uniform((100, 1), 0.0, 1.0)

# Define the model
model = PINN()

# Define the thermal diffusivity constant
alpha = 0.01

# Define the target time and the desired state at that time
t_target = tf.constant([[1.0]], dtype=tf.float32)
u_target = tf.constant(np.sin(np.pi * np.linspace(0, 1, 100)).reshape(-1, 1) * np.exp(-alpha * np.pi**2), dtype=tf.float32)

# Define the initial condition (as a trainable variable)
x0 = tf.linspace(0.0, 1.0, 100)[:, tf.newaxis]
u0 = initial_condition(x0)

# Train the model to find the optimal initial condition
train_optimal_control(model, x_train, t_train, x0, u0, alpha, u_target, t_target)

# Validate the solution
def validate_optimal_solution(model, x0, u0, u_target, alpha):
    x_test = tf.linspace(0.0, 1.0, 100)[:, tf.newaxis]
    t_test = tf.constant([[1.0]], dtype=tf.float32)
    t_test_broadcast = tf.broadcast_to(t_test, tf.shape(x_test))
    u_pred = model(x_test, t_test_broadcast)
    u_pred = tf.reshape(u_pred, (100, 1))

    plt.plot(x0, u_pred, label='Prediction at t=1.0')
    plt.plot(x0, u_target, label='Target State at t=1.0', linestyle='--')
    plt.plot(x0, u0.numpy(), label='Optimized Initial Condition', linestyle='-.')  # Convert u0 to numpy for plotting
    plt.legend()
    plt.show()

# Validate the result
validate_optimal_solution(model, x0, u0, u_target, alpha)
